In [1]:
#데이터셋 불러오고 X, Y 정하기
import numpy as np
import pandas as pd

# ANN으로 한 번, 커널로 한 번, 트리나 xgb로 한 번, 다른 거 추가할 수 있으면 추가

df = pd.read_csv("/content/drive/MyDrive/Inhibitor classification/novdataset/PA_nov_Molprint2.csv", index_col=None)
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

seed=0
np.random.seed(seed)

In [ ]:
#feature 추리기

#편차 거의 없는 거 제외하기
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
X = sel.fit_transform(X)

#본격적인 피처 추리기
from sklearn.feature_selection import GenericUnivariateSelect, f_classif

transformer = GenericUnivariateSelect(f_classif, mode='percentile', param=20)
X_new = transformer.fit_transform(X, Y)
X_new.shape


(115, 205)

In [3]:
#train, testset 나누기
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler, MaxAbsScaler,  QuantileTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

train_input, test_input, train_target, test_target = train_test_split(X, Y, test_size=0.2, random_state=seed)
#정확도 어느 정도 올라가면 train이랑 test set 별도로 저장하기

In [2]:
#층화임의추출

from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in sss.split(X, Y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

In [3]:
#Train만 oversampling
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=seed)
X_over, Y_over = smote.fit_resample(X_train, Y_train)

In [ ]:
# SVC로 분류해보기
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler, MaxAbsScaler,  QuantileTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.utils.fixes import loguniform
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.combine import SMOTEENN

#smote = SMOTEENN(random_state=seed)
#X_over, Y_over = smote.fit_resample(X_train, Y_train)

#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

#train_input, test_input, train_target, test_target = train_test_split(X, Y, test_size=0.2, random_state=seed)

param_grid = [{'kernel':['poly','rbf','linear','sigmoid'],  'C': loguniform(1, 1e3), 'gamma': loguniform(1e-3, 1)}]
grid =RandomizedSearchCV(SVC(probability=True, random_state=seed), param_grid, n_jobs=-1, n_iter=5000, cv=5, scoring='accuracy')
svc = SVC(probability=True, random_state=seed)
grid.fit(X_over, Y_over)

pred_train = grid.predict(X_over)
pred_test = grid.predict(test_input)
pred_total = grid.predict(X)
accuracy = accuracy_score(Y_over, pred_train)
ex_accuracy = accuracy_score(test_target, pred_test)
tot_accuracy = accuracy_score(Y, pred_total)

print(grid.best_params_)

print('SVM train 정확도: {0:.4f}'.format(accuracy))
print('SVM test 정확도: {0:.4f}'.format(ex_accuracy))
print('SVM total 정확도: {0:.4f}'.format(tot_accuracy))

conf = confusion_matrix(Y_over, pred_train, labels=[0,1,2,3])
exconf = confusion_matrix(test_target, pred_test, labels=[0,1,2,3])
totconf = confusion_matrix(Y, pred_total, labels=[0,1,2,3])

print(conf)
print(exconf)
print(totconf)

#모델 저장 
import joblib

svm = grid.best_estimator_

saved_model = joblib.dump(svm,'/content/drive/MyDrive/Inhibitor classification/novdataset/PA_SVM_1203_molprint2_over_tune05.pkl')

{'C': 4.685334837751994, 'gamma': 0.17213362209787414, 'kernel': 'rbf'}
SVM train 정확도: 0.9521
SVM test 정확도: 0.8696
SVM total 정확도: 0.8783
[[46  0  0  1]
 [ 0 47  0  0]
 [ 0  0 40  7]
 [ 0  0  1 46]]
[[ 5  0  0  0]
 [ 1 12  0  0]
 [ 0  0  2  0]
 [ 0  2  0  1]]
[[27  5  0  0]
 [ 4 55  0  0]
 [ 0  2  9  0]
 [ 0  3  0 10]]


In [ ]:
#xgboost
import xgboost as xgb

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform, loguniform

xgb = XGBClassifier()
params = {'max_depth': list(range(2,20)),'learning_rate': uniform(1e-5, 0.3),'subsample': uniform(0.1, 1), 
          'min_child_weight': uniform(0.5, 10),'gamma': uniform(0, 1),'n_estimators': list(range(500, 2000))}

xgb_wrapper = RandomizedSearchCV(xgb, params, n_iter = 100, n_jobs=-1, cv=5)

xgbs = xgb_wrapper.fit(X_over, Y_over)

best_params = xgbs.best_params_
print(best_params)
xgbs1 = xgbs.best_estimator_
#xgbs1 = xgb.fit(X_over, Y_over)
pred = xgbs1.predict(X_over)
accuracy = accuracy_score(Y_over, pred)
print('xgb train 정확도: {0:.4f}'.format(accuracy))
conf_matrix = confusion_matrix(Y_over, pred)
print(conf_matrix)

expred = xgbs1.predict(X_test)
accuracy = accuracy_score(Y_test, expred)
print('xgb test 정확도: {0:.4f}'.format(accuracy))
ex_conf_matrix = confusion_matrix(Y_test, expred)
print(ex_conf_matrix)

totpred = xgbs1.predict(X)
accuracy = accuracy_score(Y, totpred)
print('xgb 전체 정확도: {0:.4f}'.format(accuracy))
tot_conf_matrix = confusion_matrix(Y, totpred)
print(tot_conf_matrix)


import joblib

saved_model = joblib.dump(xgbs1,'/content/drive/MyDrive/Inhibitor classification/novdataset/PA_XGB_1229_over_tune04.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py", line 732, in fit
    callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/training.py", line 216, in train
    xgb_model=xgb_model, callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/training.py", line 

{'gamma': 0.16897605441274532, 'learning_rate': 0.11907525266549945, 'max_depth': 17, 'min_child_weight': 0.7454811652397759, 'n_estimators': 1882, 'subsample': 0.4112467981437029}
xgb train 정확도: 0.9681
[[46  0  0  1]
 [ 0 47  0  0]
 [ 0  0 42  5]
 [ 0  0  0 47]]
xgb test 정확도: 0.3478
[[3 3 0 0]
 [5 4 1 2]
 [0 1 0 1]
 [0 1 1 1]]
xgb 전체 정확도: 0.8696
[[29  3  0  0]
 [ 5 51  1  2]
 [ 0  1  9  1]
 [ 0  1  1 11]]


In [5]:
#RF

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

params = {'n_estimators': list(range(10,1000)), 'criterion': ['gini', 'entropy'], 'max_depth': list(range(2,10)), 
          'min_samples_split': list(range(2,13)), 'min_samples_leaf': list(range(1,10)), 'random_state': list(range(1,100))}

rf = RandomForestClassifier()
#rfs1 = rf.fit(X_over, Y_over)

rf_wrapper = RandomizedSearchCV(rf, params, n_iter = 1000, n_jobs=-1, cv=5)

rfs = rf_wrapper.fit(X_over, Y_over)

best_params = rfs.best_params_
print(best_params)
rfs1 = rfs.best_estimator_

pred = rfs1.predict(X_over)
accuracy = accuracy_score(Y_over, pred)
print('rf train 정확도: {0:.4f}'.format(accuracy))
conf_matrix = confusion_matrix(Y_over, pred)
print(conf_matrix)

expred = rfs1.predict(X_test)
accuracy = accuracy_score(Y_test, expred)
print('rf test 정확도: {0:.4f}'.format(accuracy))
ex_conf_matrix = confusion_matrix(Y_test, expred)
print(ex_conf_matrix)

totpred = rfs1.predict(X)
accuracy = accuracy_score(Y, totpred)
print('rf 전체 정확도: {0:.4f}'.format(accuracy))
tot_conf_matrix = confusion_matrix(Y, totpred)
print(tot_conf_matrix)


import joblib

saved_model = joblib.dump(rfs1,'/content/drive/MyDrive/Inhibitor classification/novdataset/PA_RF_0113_overtune05.pkl')

{'random_state': 7, 'n_estimators': 22, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_depth': 9, 'criterion': 'gini'}
rf train 정확도: 0.8777
[[40  1  3  3]
 [ 4 41  0  2]
 [ 0  0 38  9]
 [ 0  0  1 46]]
rf test 정확도: 0.3913
[[3 1 1 1]
 [3 3 2 4]
 [1 0 0 1]
 [0 0 0 3]]
rf 전체 정확도: 0.8000
[[27  2  2  1]
 [ 7 44  2  6]
 [ 1  0  8  2]
 [ 0  0  0 13]]


In [ ]:
#모델을 잘 만들어도 set 내부의 과적합 문제를 해결하지 못하면 의미가 없다.
#과적합 해결해야 함

In [ ]:
#LOOCV 시험
import xgboost as xgb

from numpy import mean
from numpy import std
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier # 0.4 중반
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier # 0.4 후반
from sklearn.linear_model import LogisticRegression # 0.47
from sklearn.svm import SVC #0.53
from lightgbm import LGBMClassifier #0.48

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform, loguniform

# create loocv procedure
cv = LeaveOneOut()
# create model
model = LGBMClassifier(random_state=seed)
# evaluate model
scores = cross_val_score(model, X_scaled, Y, cv=cv, n_jobs=-1)
# report performance
print('Score: %.3f (%.3f)' % (mean(scores), std(scores)))
#print(cv.get_n_splits(X_scaled))
#print(cv.split(X_scaled))


Score: 0.478 (0.500)


In [ ]:
#데이터 입력
df = pd.read_csv('/content/drive/MyDrive/Inhibitor classification/Inhibitor/PA/MORDRED_2D/PA_morgan_2D_apppendscore_dropsmiles.csv')
data = pd.read_csv('/content/drive/MyDrive/Inhibitor classification/210518 dataset/PA/PA_descriptors_drop_SMILES.csv')

#데이터 분류
ref = df.values
X_ref = ref[:,0:-1]
Y_ref = ref[:,-1]

dataset = data.values
X = dataset

#X 표준화

scaler = StandardScaler()

X_ref_scaled = scaler.fit_transform(X_ref)
X_scaled = scaler.transform(X)

#load model

loaded_model = joblib.load('/content/drive/MyDrive/Inhibitor classification/210518 dataset/PA/PA_SVC_0412_02.pkl')

#모델로 예측

pred = loaded_model.predict(X_scaled)
result = pd.DataFrame({'class' : pred})
complete = pd.concat([data, result], axis=1)
complete
complete.to_csv('/content/drive/MyDrive/Inhibitor classification/210518 dataset/PA/PA_result.csv')